In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import h5py

from plot import PlotAx, STYLE_PATH
from data_loader import LoaderCrisL1

%matplotlib inline

/home/cali/bin/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:38: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [3]:
in_dir = '/home/cali/data/GapFilling/CRISFull/'
filenames = os.listdir(in_dir)
in_files = [os.path.join(in_dir, filename) for filename in filenames]
in_files.sort()

# ranges_X = [(650., 1095.), (1210., 1750.), (2155., 2550.)]
# ranges_Y = [(1095.625, 1209.375), (1750.625, 2154.375), (2550.625, 2755.)]

ranges_X = [(650., 1095), ]
ranges_Y = [(1095.625, 1209.375), ]

# ranges_X = [(1210., 1750), ]
# ranges_Y = [(1750.625, 2154.375), ]

# ranges_X = [(2155., 2550.)]
# ranges_Y = [(2550., 2755.)]

from util import get_cris_full_train_data, get_range_index
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA

import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

In [4]:
count = None # 加载全部的数据
X, Y = get_cris_full_train_data(in_files, x_ranges=ranges_X, y_ranges=ranges_Y, count=count)
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, random_state = 42)

In [14]:
train_count = 160000
test_count = 40000

train_X_count = train_X.iloc[:train_count]
test_X_count = test_X.iloc[:test_count]

train_Y_count = train_Y.iloc[:train_count, :10]
test_Y_count = test_Y.iloc[:test_count, :10]

In [15]:
train_Y_count.head()

,0,1,2,3,4,5,6,7,8,9
498733,21.431185,21.268238,21.023870,20.970291,21.021344,20.951293,20.635818,20.448134,20.637161,20.510014
55366,27.891033,28.173353,28.083853,27.924261,27.759690,27.867721,27.855893,27.453846,27.335153,27.460305
493668,44.196197,44.441536,44.329308,44.294757,44.035717,43.823831,43.348396,42.801220,43.218996,42.664302
619158,34.373630,34.560009,34.563323,34.319419,34.311104,34.136819,33.694970,33.520200,33.789269,33.745341
761325,9.943563,9.940375,9.914521,9.905382,9.839935,9.843989,9.741329,9.851226,9.967051,9.759227


In [7]:
pca_component_n = 100
pca = PCA(pca_component_n)
pca.fit(train_X_count)
train_X_count_pca = pca.transform(train_X_count)
test_X_count_pca = pca.transform(test_X_count)

In [16]:
train_X_count_pca = train_X_count
test_X_count_pca = test_X_count

In [17]:
test_X_count_pca.shape

(40000, 713)

In [18]:
param = {}
param['objective'] = 'reg:linear'
param['eval_metric'] = 'mae'
param['tree_method'] = 'gpu_hist'
param['silent'] = 1

# evallist = [(dtest, 'eval'), (dtrain, 'train')]

kw = {'tree_method': 'gpu_hist', 'n_gpus': 1, 'max_bin': 16, 'silent': False}  # 使用 GPU 训练

xgb_reg_multi = MultiOutputRegressor(xgb.XGBRegressor(max_depth=3, objective='reg:linear', **kw)).fit(train_X_count_pca, train_Y_count)
# xgb_reg = xgb.XGBRegressor(max_depth=3, objective='reg:linear', n_estimators=500, **kw).fit(train_X_count_pca, train_Y_count)

KeyboardInterrupt: 

In [ ]:
Y_predictions = xgb_reg_multi.predict(test_X_count_pca)

mae = mean_absolute_error(test_Y_count, Y_predictions)
print(mae)